In [16]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString

In [17]:
# Modified trips.txt with Excel to fix some issues regarding some shape allocations
trips_df = pd.read_excel("gtfs/trips_fixed.xlsx", dtype=str)
#trips_df = pd.read_csv("gtfs/trips.txt", dtype=str, low_memory=False)

# choose one trip per shape_id preferring service_id in order: 'LD' (highest), then 'LS', then 'LV'
# uses existing variables: selected_trips (DataFrame) and priority_map (dict)
# define priority for service_id values (lower is higher priority)
priority_map = {'LD': 0, 'LS': 1, 'LV': 2}

# work on a copy of trips_df
tmp = trips_df.copy()
tmp['service_prio'] = tmp['service_id'].map(priority_map).fillna(999).astype(int)

# sort by shape_id then priority so the preferred service is first; drop duplicates to keep the chosen trip per shape
tmp = tmp.sort_values(['shape_id', 'service_prio'])
rep = tmp.drop_duplicates(subset='shape_id', keep='first').reset_index(drop=True)

# DataFrame with the assignments
transit_df = rep[['shape_id', 'route_id', 'trip_id', 'trip_headsign']]

# Remove route "T14B" since it has problems with its shape
transit_df = transit_df[transit_df['route_id'] != 'T14B']

print(f"Assigned route_id for {len(transit_df)} shapes.")
print(transit_df.head())
print(transit_df.info())

Assigned route_id for 486 shapes.
  shape_id route_id                    trip_id          trip_headsign
0   C01_r1      C01                 C01_trip_1   Centro Metropolitano
1   C01_r2      C01                 C01_trip_2     San Juan de Ocotan
2   C02_r1      C02                 C02_trip_1   Centro Metropolitano
3   C02_r2      C02                 C02_trip_2  Mision de los Viñedos
4   C03_r1      C03  C03_route_1_1111110_18000              El Fresno
<class 'pandas.core.frame.DataFrame'>
Index: 486 entries, 0 to 487
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   shape_id       486 non-null    object
 1   route_id       486 non-null    object
 2   trip_id        486 non-null    object
 3   trip_headsign  486 non-null    object
dtypes: object(4)
memory usage: 19.0+ KB
None


In [18]:
# Read GTFS stop_times.txt; read everything as string to avoid dtype surprises
stop_times_df = pd.read_csv("gtfs/stop_times.txt", dtype=str, low_memory=False)

# filter by trip_ids in transit_df
stop_times_df = stop_times_df[stop_times_df['trip_id'].isin(transit_df['trip_id'])]

# Convert departure times to pandas Timedelta (handles >24:00:00)
stop_times_df["departure_time"] = pd.to_timedelta(stop_times_df["departure_time"], errors="coerce")
stop_times_df["stop_sequence"] = pd.to_numeric(stop_times_df["stop_sequence"], errors="coerce")

# Quick check
print(stop_times_df.info())
print(stop_times_df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 24357 entries, 80 to 48901
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   trip_id         24357 non-null  object         
 1   stop_id         24357 non-null  object         
 2   stop_sequence   24357 non-null  int64          
 3   stop_headsign   24357 non-null  object         
 4   arrival_time    24357 non-null  object         
 5   departure_time  24357 non-null  timedelta64[ns]
 6   timepoint       24357 non-null  object         
dtypes: int64(1), object(5), timedelta64[ns](1)
memory usage: 1.5+ MB
None
       trip_id           stop_id  stop_sequence stop_headsign arrival_time  \
80  C01_trip_1  mxc_C01_P_STP_01              0      Libertad     04:35:00   
81  C01_trip_1  mxc_C01_P_STP_02              1  Calle Ocampo     04:36:32   
82  C01_trip_1  mxc_C01_P_STP_03              2  Calle Parral     04:37:18   
83  C01_trip_1  mxc_C01_P_STP_04

In [19]:
# Aggregate stop_times by trip_id to create lists of stop_ids, stop_headsigns, and time deltas between consecutive stops
def _condense_trip_info(g):
    g = g.copy()
    g = g.sort_values('stop_sequence')
    stops = g['stop_id'].tolist()
    stop_headsigns = g['stop_headsign'].tolist()
    times = g['departure_time']
    deltas = times.diff().iloc[1:].dt.total_seconds().astype(int).tolist()  # list of int seconds (length = len(stops)-1)
    time_trip = int((times.iloc[-1] - times.iloc[0]).total_seconds())  # total trip time in seconds
    return pd.Series({'stop_ids': stops, 'stop_headsigns': stop_headsigns, 'stop_time_deltas': deltas, 'time_trip': time_trip, 'num_stops': len(stops)})

transit_stop_df = stop_times_df.groupby('trip_id').apply(_condense_trip_info).reset_index()

# quick check
print(transit_stop_df.head())
print(transit_stop_df.info())

   trip_id                                           stop_ids  \
0    A03_1  [MM_A03_1, MM_A03_2, MM_A03_3, MM_A03_4, MM_A0...   
1    A03_2  [MM_A03_17, MM_A03_19, MM_A03_20, MM_C138_47, ...   
2    A05_1  [MM_A05_26, MM_A05_2, MM_A05_3, MM_A05_4, MM_A...   
3    A05_2  [MM_A05_14, MM_A05_15, MM_A05_16, MM_A05_17, M...   
4  A06_1_1  [MM_A06_1, MM_A06_2, MM_A06_4, MM_A06_5, MM_A0...   

                                      stop_headsigns  \
0  [Terminal de la Ruta 110, Avenida Antonio Diaz...   
1  [Avenida Experiencia, Avenida Experiencia, Ave...   
2  [Avenida Belisario Domínguez, Francisco Sosa, ...   
3  [Calzada Independencia Norte, Comisión Federal...   
4  [Calle Río Blanco, Calle Rio Colorado, Río Tux...   

                                    stop_time_deltas  time_trip  num_stops  
0  [74, 60, 49, 66, 57, 47, 49, 49, 47, 47, 46, 5...        849         17  
1  [95, 53, 55, 49, 51, 47, 47, 48, 49, 49, 47, 5...        942         18  
2  [50, 61, 51, 57, 43, 44, 86, 60, 55, 4

C:\Users\PEREJA12\AppData\Local\Temp\ipykernel_38600\3912352740.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  transit_stop_df = stop_times_df.groupby('trip_id').apply(_condense_trip_info).reset_index()


In [20]:
stop_df = pd.read_csv("gtfs/stops.txt", dtype=str, low_memory=False)
#filter the stop_df to only include stops in transit_stop_df
stop_df = stop_df[stop_df['stop_id'].isin(transit_stop_df['stop_ids'].explode())]

# Add to the transit_stop_df the Linestring geometry of each stop
stop_df['stop_lat'] = pd.to_numeric(stop_df['stop_lat'], errors='coerce')
stop_df['stop_lon'] = pd.to_numeric(stop_df['stop_lon'], errors='coerce')

#for each trip in transit_stop_df, create a linestring geometry of its stops
def _create_stop_linestring(row):
    stop_ids = row['stop_ids']
    coords = []
    for stop_id in stop_ids:
        stop_info = stop_df[stop_df['stop_id'] == stop_id]
        if not stop_info.empty:
            lat = stop_info['stop_lat'].values[0]
            lon = stop_info['stop_lon'].values[0]
            coords.append((lat, lon))  # Note: (lat, lon) for Point
    if len(coords) == 0:
        return None
    elif len(coords) == 1:
        return Point(coords[0])
    else:
        return LineString(coords)
    
transit_stop_df['stop_geometry'] = transit_stop_df.apply(_create_stop_linestring, axis=1)

# quick check
print(transit_stop_df.head())
print(transit_stop_df.info())

   trip_id                                           stop_ids  \
0    A03_1  [MM_A03_1, MM_A03_2, MM_A03_3, MM_A03_4, MM_A0...   
1    A03_2  [MM_A03_17, MM_A03_19, MM_A03_20, MM_C138_47, ...   
2    A05_1  [MM_A05_26, MM_A05_2, MM_A05_3, MM_A05_4, MM_A...   
3    A05_2  [MM_A05_14, MM_A05_15, MM_A05_16, MM_A05_17, M...   
4  A06_1_1  [MM_A06_1, MM_A06_2, MM_A06_4, MM_A06_5, MM_A0...   

                                      stop_headsigns  \
0  [Terminal de la Ruta 110, Avenida Antonio Diaz...   
1  [Avenida Experiencia, Avenida Experiencia, Ave...   
2  [Avenida Belisario Domínguez, Francisco Sosa, ...   
3  [Calzada Independencia Norte, Comisión Federal...   
4  [Calle Río Blanco, Calle Rio Colorado, Río Tux...   

                                    stop_time_deltas  time_trip  num_stops  \
0  [74, 60, 49, 66, 57, 47, 49, 49, 47, 47, 46, 5...        849         17   
1  [95, 53, 55, 49, 51, 47, 47, 48, 49, 49, 47, 5...        942         18   
2  [50, 61, 51, 57, 43, 44, 86, 60, 55

In [21]:
frequencies_df = pd.read_csv("gtfs/frequencies.txt", dtype=str, low_memory=False)
# filter the frequencies_df to only include trips in transit_df
frequencies_df = frequencies_df[frequencies_df['trip_id'].isin(transit_df['trip_id'])]


# Convert headway_secs to numeric
frequencies_df['frequency'] = pd.to_numeric(frequencies_df['headway_secs'], errors='coerce')
# Combine to have only one entry per trip_id, taking the average headway_secs if multiple entries exist
frequencies_df = frequencies_df.groupby('trip_id')['frequency'].mean().astype(int).reset_index()
   
# quick check
print(frequencies_df.head())
print(frequencies_df.info())

   trip_id  frequency
0    A03_1        900
1    A03_2        900
2    A05_1       1320
3    A05_2       1320
4  A06_1_1        756
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   trip_id    486 non-null    object
 1   frequency  486 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.7+ KB
None


In [22]:
shapes_df = pd.read_csv("gtfs/shapes.txt", dtype=str, low_memory=False)
routes_df = pd.read_csv("gtfs/routes.txt", dtype=str, low_memory=False)

In [23]:
# To verify which shapes and routes are used in transit_df

set_shapes = set(shapes_df['shape_id'])
set_routes = set(routes_df['route_id'])

set_used_shapes = set(transit_df['shape_id'])
set_used_routes = set(transit_df['route_id'])

#these should be empty sets
print(set_used_shapes - set_shapes)
print(set_used_routes - set_routes)

#these are the shapes (routes with directions) that wont be used
print(set_shapes - set_used_shapes)
print(set_routes - set_used_routes)

set()
set()
{'T13B-V1_r2', 'T14B_r2', 'T07-C01_r1', 'T15-C06_r1', 'T15-C02_r2', 'T13B-C02-1_r1', 'T17-1_r1', 'T13A-C03_r1', 'T06-03_r2', 'T15-1_r2', 'T16A-C01_r2', 'T13A-C02_r2', 'T07-C04_r2', 'T15-C01_r1', 'T14A-C01_r1', 'T13C-2_r2', 'T17-1_r2', 'T16B-C03_r2', 'T13B-V2_r2', 'T13C-2_r1', 'T13A-C02_r1', 'T13B-V1_r1', 'C63_r2', 'T15-C03_r1', 'T13A-C03_r2', 'T15-1_r1', 'T15-C05_r2', 'T14A-C03_r2', 'T14A-C01_r2', 'T13A-C01_r2', 'T16A-C01_r1', 'T15-C04_r2', 'T13C-1_r2', 'T15-C03_r2', 'T16B-1_r2', 'T15-C02_r1', 'T16B-C01_r2', 'T15-C01_r2', 'T13B-C01-1_r2', 'T15-C06_r2', 'T15-C05_r1', 'T13C-1_r1', 'T14B_r1', 'T13B-V2_r1', 'T16B-C01_r1', 'T16B-C03_r1', 'T16B-1_r1', 'T15-C04_r1', 'T07-C01_r2', 'T07-C04_r1', 'T14A-C02_r1', 'T13B-C01-1_r1', 'T14A-C02_r2', 'T06-03_r1', 'T13A-C01_r1', 'T13B-C02-1_r2', 'C119_r2', 'T14A-C03_r1', 'C119_r1'}
{'T16B-C03', 'T14A-C02', 'T15-C01', 'T13B-V1', 'T15-C03', 'T07-C01', 'T13B-V2', 'T13A-C01', 'T15-C06', 'T15-C05', 'T14B', 'T16B-C01', 'T13B-C01-1', 'T15-1', 'T15-C

In [24]:
# Convert shapes_df into a GeoDataFrame of LineStrings (one per shape_id)

# filter by shape_id in transit_df
shapes_df = shapes_df[shapes_df['shape_id'].isin(transit_df['shape_id'])]

# Ensure numeric columns and drop invalid rows
shapes_df['shape_pt_lat'] = pd.to_numeric(shapes_df['shape_pt_lat'], errors='coerce')
shapes_df['shape_pt_lon'] = pd.to_numeric(shapes_df['shape_pt_lon'], errors='coerce')
shapes_df['shape_pt_sequence'] = pd.to_numeric(shapes_df['shape_pt_sequence'], errors='coerce')

shapes_pts = shapes_df.dropna(subset=['shape_pt_lat','shape_pt_lon','shape_pt_sequence']).copy()
shapes_pts = shapes_pts.sort_values(['shape_id','shape_pt_sequence'])

# Build LineString per shape_id
lines = []
for shape_id, grp in shapes_pts.groupby('shape_id'):
    grp = grp.sort_values('shape_pt_sequence')
    coords = [(x, y) for x, y in zip(grp['shape_pt_lon'], grp['shape_pt_lat'])]
    if len(coords) == 0:
        continue
    geom = LineString(coords) if len(coords) > 1 else Point(coords[0])
    lines.append({'shape_id': shape_id, 'shape_geometry': geom, 'shape_pt_count': len(coords)})

shapes_df = pd.DataFrame(lines)

first_rows = shapes_pts.sort_values('shape_pt_sequence').groupby('shape_id').first().reset_index()
shapes_df = shapes_df.merge(first_rows.drop(columns=['shape_pt_lat','shape_pt_lon','shape_pt_sequence']), on='shape_id', how='left')

print('Number of shapes created:', len(shapes_df))
shapes_df.head()
shapes_df.info()

Number of shapes created: 486
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   shape_id        486 non-null    object
 1   shape_geometry  486 non-null    object
 2   shape_pt_count  486 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 11.5+ KB


In [25]:

# filter by route_id in transit_df
routes_df = routes_df[routes_df['route_id'].isin(transit_df['route_id'])]

# Add "#" to the colors of the routes_df
routes_df['route_color'] = routes_df['route_color'].apply(lambda c: f"#{c}" if pd.notna(c) and not c.startswith('#') else c)

#to numeric route_type
routes_df['route_type'] = pd.to_numeric(routes_df['route_type'], errors='coerce')

# Modify route_type for the Macrobus (Troncales only: MC-L1, MC-L1E, MP-T01, MP-T02, MP-T03)
routes_df.loc[routes_df['route_short_name'].isin(["MC-L1", "MC-L1E", "MP-T01", "MP-T02", "MP-T03"]), 'route_type'] = 1

In [ ]:
# Add to the transit_df the geometry of each shape_id
transit_df = transit_df.merge(shapes_df[['shape_id', 'shape_geometry']], on='shape_id', how='left')
transit_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   shape_id        486 non-null    object
 1   route_id        486 non-null    object
 2   trip_id         486 non-null    object
 3   trip_headsign   486 non-null    object
 4   shape_geometry  486 non-null    object
dtypes: object(5)
memory usage: 19.1+ KB


In [27]:
# Add to the transit_df the information of each route_id

transit_df = transit_df.merge(routes_df[["route_id", "route_short_name", "route_long_name", "route_type", "route_color"]], on='route_id', how='left')
transit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   shape_id          486 non-null    object
 1   route_id          486 non-null    object
 2   trip_id           486 non-null    object
 3   trip_headsign     486 non-null    object
 4   shape_geometry    486 non-null    object
 5   route_short_name  486 non-null    object
 6   route_long_name   486 non-null    object
 7   route_type        486 non-null    int64 
 8   route_color       486 non-null    object
dtypes: int64(1), object(8)
memory usage: 34.3+ KB


In [28]:
# Add to the transit_df the information of stops per trip_id

transit_df = transit_df.merge(transit_stop_df, on='trip_id', how='left')
transit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   shape_id          486 non-null    object
 1   route_id          486 non-null    object
 2   trip_id           486 non-null    object
 3   trip_headsign     486 non-null    object
 4   shape_geometry    486 non-null    object
 5   route_short_name  486 non-null    object
 6   route_long_name   486 non-null    object
 7   route_type        486 non-null    int64 
 8   route_color       486 non-null    object
 9   stop_ids          486 non-null    object
 10  stop_headsigns    486 non-null    object
 11  stop_time_deltas  486 non-null    object
 12  time_trip         486 non-null    int64 
 13  num_stops         486 non-null    int64 
 14  stop_geometry     486 non-null    object
dtypes: int64(3), object(12)
memory usage: 57.1+ KB


In [29]:
# Add to the transit_df the information of frequencies per trip_id

transit_df = transit_df.merge(frequencies_df[["trip_id", "frequency"]], on='trip_id', how='left')
transit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   shape_id          486 non-null    object
 1   route_id          486 non-null    object
 2   trip_id           486 non-null    object
 3   trip_headsign     486 non-null    object
 4   shape_geometry    486 non-null    object
 5   route_short_name  486 non-null    object
 6   route_long_name   486 non-null    object
 7   route_type        486 non-null    int64 
 8   route_color       486 non-null    object
 9   stop_ids          486 non-null    object
 10  stop_headsigns    486 non-null    object
 11  stop_time_deltas  486 non-null    object
 12  time_trip         486 non-null    int64 
 13  num_stops         486 non-null    int64 
 14  stop_geometry     486 non-null    object
 15  frequency         486 non-null    int64 
dtypes: int64(4), object(12)
memory usage: 60.9+ KB


In [30]:
transit_df.to_pickle("gtfs/transit_df.pkl")
transit_df.to_excel("gtfs/transit_df.xlsx")


In [32]:
transit_stop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   trip_id           486 non-null    object
 1   stop_ids          486 non-null    object
 2   stop_headsigns    486 non-null    object
 3   stop_time_deltas  486 non-null    object
 4   time_trip         486 non-null    int64 
 5   num_stops         486 non-null    int64 
 6   stop_geometry     486 non-null    object
dtypes: int64(2), object(5)
memory usage: 26.7+ KB


In [33]:
from gtfs_loader import load_transit_dataframe

transit2_df = load_transit_dataframe("gtfs")

c:\Users\PEREJA12\OneDrive - ITESO\Maestría en Ciencias Computacionales\Análisis y Diseño de Algoritmos\project\map_app\data\gtfs_loader.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times_df["departure_time"] = pd.to_timedelta(stop_times_df["departure_time"], errors="coerce")
c:\Users\PEREJA12\OneDrive - ITESO\Maestría en Ciencias Computacionales\Análisis y Diseño de Algoritmos\project\map_app\data\gtfs_loader.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times_df["stop_sequence"]

In [34]:
transit2_df.to_excel("gtfs/transit2_df.xlsx")